# Juan Manuel Deutsch, Cesar Felipe Giraldo, Julian Felipe Pulido

# Exercise 1

Para este ejercicio tenemos $X$ siendo la matriz base y $Σ$ como la matriz de covarianza de $X$

Tambien tenemos los componentes principales $u_1, u_2,...,u_n$, que son los autovectores o vectores propios de $Σ$ que tienen varianzas asociadas $u_i^TΣu_i$, donde $u_i^T$ es la transpuesta del vector propio $u_i$

Ahora tendremos una tranformación lineal de los datos $X$, siendo $AX$ donde $A$ es una transformación $m x m$

Ahora con la matriz transformada tenemos la matriz de covarianza transformada $AΣA^T$ cuyos componentes principales estan dados por $v_1, v_2,...,v_n$ estos siendo los autovectores cuyas varianza asociadas estan dadas por $v_i^TAΣA^Tv_i$

# Exercise 2

In [9]:
import numpy as np
D = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
red = 1
def PCA(D, red):

    mu = list(sum(row)/len(row) for row in zip(*D))
    n = D.shape[0]
    centeredData = D - mu
    cov = np.dot(centeredData.T, centeredData) / (n)
    eigenValues, eigenVectors = np.linalg.eig(cov)
    print(eigenVectors)
    if(red < 1):
        print('lo que sea')
    else: 
        r = red
    Ur = eigenVectors[:, :r]
    print(Ur)

PCA(D, red)


[[ 5.77350269e-01 -3.27465273e-18  8.09500438e-01]
 [ 5.77350269e-01 -7.07106781e-01 -4.97117858e-01]
 [ 5.77350269e-01  7.07106781e-01 -3.12382580e-01]]
[[0.57735027]
 [0.57735027]
 [0.57735027]]
